# Import

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install torchsummaryX wandb --quiet

In [ ]:
import wandb

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import sys
sys.path.append('/content/gdrive/MyDrive/AD_NET/AD-Net/sai_stuff/Code/vit')

In [ ]:
!pip install einops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install pydicom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install SimpleITK

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import sys 
import json
import glob
import random
import collections
import time
import re

import numpy as np
import pandas as pd
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch import nn
from torch.utils import data as torch_data
from sklearn import model_selection as sk_model_selection
from torch.nn import functional as torch_functional
import torch.nn.functional as F

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

key = "key"

#Weights and Bias
if key:
  wandb.login(key=key) #API Key is in your wandb account, under settings (wandb.ai/settings)


Device:  cuda


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: saisravjo (stack-it). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
def set_seed(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True

set_seed(12)

In [ ]:
import sys
sys.path.append('/content/gdrive/MyDrive/AD_NET/AD-Net/sai_stuff/vit')

In [ ]:
from model import Model
from ad_net_dataset import ADNetDataset
from trainer import Trainer

In [ ]:
train_dataset = ADNetDataset('/content/gdrive/MyDrive/AD_NET/AD-Net/sai_stuff/vit/data/ad_train_ds', '/content/gdrive/MyDrive/AD_NET/AD-Net/sai_stuff/vit/data/cn_train_ds')
test_dataset = ADNetDataset('/content/gdrive/MyDrive/AD_NET/AD-Net/sai_stuff/vit/data/ad_test_ds', '/content/gdrive/MyDrive/AD_NET/AD-Net/sai_stuff/vit/data/cn_test_ds')

(64, 128, 128) /content/gdrive/MyDrive/AD_NET/AD-Net/sai_stuff/vit/data/ad_train_ds/ADNI_127_S_0844_reg_brain.nii
(64, 128, 128) /content/gdrive/MyDrive/AD_NET/AD-Net/sai_stuff/vit/data/ad_train_ds/ADNI_126_S_0606_reg_brain.nii
(64, 128, 128) /content/gdrive/MyDrive/AD_NET/AD-Net/sai_stuff/vit/data/ad_train_ds/ADNI_116_S_0487_reg_brain.nii
(64, 128, 128) /content/gdrive/MyDrive/AD_NET/AD-Net/sai_stuff/vit/data/ad_train_ds/ADNI_116_S_0392_reg_brain.nii
(64, 128, 128) /content/gdrive/MyDrive/AD_NET/AD-Net/sai_stuff/vit/data/ad_train_ds/ADNI_067_S_1253_reg_brain.nii
(64, 128, 128) /content/gdrive/MyDrive/AD_NET/AD-Net/sai_stuff/vit/data/ad_train_ds/ADNI_032_S_1101_reg_brain.nii
(64, 128, 128) /content/gdrive/MyDrive/AD_NET/AD-Net/sai_stuff/vit/data/ad_train_ds/ADNI_031_S_1209_reg_brain.nii
(64, 128, 128) /content/gdrive/MyDrive/AD_NET/AD-Net/sai_stuff/vit/data/ad_train_ds/ADNI_027_S_1385_reg_brain.nii
(64, 128, 128) /content/gdrive/MyDrive/AD_NET/AD-Net/sai_stuff/vit/data/ad_train_ds/ADNI

In [ ]:
train_dataset[0]['X'].shape

torch.Size([1, 128, 128, 64])

# Loader

In [ ]:


config = {
    'lr': 0.01,
    'dropout': 0.1,
    'emb_dropout': 0.1,
    'threshold': 0.5,
    'num_attn_heads': 16
}

sweep_configuration = {
    'method': 'grid',
    'name': 'ViT finetuning attempt 3 grid search',
    'metric': {
        'goal': 'minimize', 
        'name': 'train_loss'
		},
    'parameters': {
        'batch_size': {'values': [30]},# Changed from 8 to 10
        'epochs': {'values': [10]},
        'patch_size': {'values': [32]},
        'threshold': {'values': [0.5]},
        'lr': {'values': [0.0001]},
        # 'threshold': {'values': [0.3, 0.4, 0.5, 0.6]},
        'num_attn_heads': {'values': [4, 8, 16, 32,]},
        # 'dropout': {'max':0.5, 'min':0.1}
     }
}


# if key:
  # Create your wandb run
  # run = wandb.init(
  #     name = "Baseline ViT", ### Wandb creates random run names if you skip this field, we recommend you give useful names
  #     reinit=True, ### Allows reinitalizing runs when you re-run this cell
  #     project="AD-Net", ### Project should be created in your wandb account 
  #     config=config ### Wandb Config for your run
  # )


# Model

In [ ]:
def main():
  torch.cuda.empty_cache()

  run = wandb.init()

  train_loader = torch_data.DataLoader(
      train_dataset,
      batch_size=wandb.config.batch_size,
      shuffle=True,
      num_workers=2,
      pin_memory = True,
  )

  test_loader = torch_data.DataLoader(
      test_dataset,
      batch_size=10,
      shuffle=True,
      num_workers=2,
      pin_memory = True,
  )

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  model = Model(
      image_size = train_dataset[0]['X'].shape[1], # 128
      patch_size = wandb.config.patch_size,
      num_classes = 1,
      dim = 1024,
      depth = 2,
      heads = wandb.config.num_attn_heads,
      mlp_dim = 2048,
      channels = 1,
      dropout = 0.1,
      emb_dropout = 0.1
  )
  model.to(device)

  optimizer = torch.optim.Adam(model.parameters(), lr=wandb.config.lr)
  criterion = torch_functional.binary_cross_entropy_with_logits

  trainer = Trainer(
      model, 
      device, 
      optimizer, 
      criterion,
      threshold=wandb.config.threshold,
  )
  history = trainer.fit(
    wandb.config.epochs, 
    train_loader, 
    test_loader, 
    '/content/gdrive/MyDrive/AD_NET/AD-Net/sai_stuff/vit/checkpoints/', 
    30,
  )

In [ ]:
sweep_id = wandb.sweep(sweep=sweep_configuration, project='AD-attnheads-3')
wandb.agent(sweep_id, function=main, count=75)

Create sweep with ID: 5txd4y54
Sweep URL: https://wandb.ai/stack-it/AD-Allin/sweeps/5txd4y54


wandb: Agent Starting Run: kxpbleeh with config:
wandb: 	batch_size: 10
wandb: 	epochs: 10
wandb: 	lr: 0.001
wandb: 	num_attn_heads: 4
wandb: 	patch_size: 32
wandb: 	threshold: 0.1
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


EPOCH: 1
Valid Step 1/1, valid_loss: 0.7044
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[Epoch Train: 1] loss: 2.3075, time: 62.00 s            
[Epoch Valid: 1] loss: 0.7044, auc: 0.5000, time: 6.00 s, precision: 0.50, recall: 1.00, f: 0.67
EPOCH: 2
Valid Step 1/1, valid_loss: 0.6929
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[Epoch Train: 2] loss: 0.7184, time: 109.00 s            
[Epoch Valid: 2] loss: 0.6929, auc: 0.5000, time: 6.00 s, precision: 0.50, recall: 1.00, f: 0.67
EPOCH: 3
Valid Step 1/1, valid_loss: 0.6970
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[Epoch Train: 3] loss: 0.8206, time: 162.00 s            
[Epoch Valid: 3] loss: 0.6970, auc: 0.5000, time: 6.00 s, precision: 0.50, recall: 1.00, f: 0.67
EPOCH: 4
Valid Step 1/1, valid_loss: 0.7129
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[Epoch Train: 4] loss: 0.7033, time: 193.00 s            
[Epoch Valid: 4] loss: 0.7129, auc: 0.5000, time: 6.00 s, precision: 0.50, recall: 1.00, f: 0.67
EPOCH: 5
Valid Step 1/1, valid_loss: 0.6945
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[Epoch

AUC,▁▁▁▁▁▁▁▁▁▁
F1-Score,▁▁▁▁▁▁▁▁▁▁
Precision,▁▁▁▁▁▁▁▁▁▁
Recall,▁▁▁▁▁▁▁▁▁▁
train_loss,█▁▂▁▁▁▁▁▁▁
AUC,0.5
F1-Score,0.66667
Precision,0.5
Recall,1.0
train_loss,0.70534


wandb: Agent Starting Run: 5eyf0oj8 with config:
wandb: 	batch_size: 10
wandb: 	epochs: 10
wandb: 	lr: 0.001
wandb: 	num_attn_heads: 4
wandb: 	patch_size: 32
wandb: 	threshold: 0.25
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Valid Step 1/1, valid_loss: 0.6905
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[Epoch Train: 5] loss: 0.7204, time: 223.00 s            
[Epoch Valid: 5] loss: 0.6905, auc: 0.5000, time: 8.00 s, precision: 0.00, recall: 0.00, f: 0.00
EPOCH: 6


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Valid Step 1/1, valid_loss: 0.7065
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[Epoch Train: 6] loss: 0.7163, time: 156.00 s            
[Epoch Valid: 6] loss: 0.7065, auc: 0.5000, time: 8.00 s, precision: 0.00, recall: 0.00, f: 0.00
EPOCH: 7


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
wandb.config

In [ ]:
# def test_():
#     with wandb.init(config=config):
#         print(wandb.config.bath_size)
#         print(wandb.config)
#         print(wandb.config.lr)

# wandb.agent(sweep_id, function=test_, count=1)